In [ ]:
from ocpy import OCP
from ocpy import DDPSolver, iLQRSolver
from ocpy import symutils

import numpy as np
import sympy as sym

# Dimensions of state and input
n_x = 2
n_u = 1
sim_name = 'lqr'

# Define ocp class
ocp = OCP(sim_name, 2, 1)

# Get symbols
x = ocp.get_x()
u = ocp.get_u()
t = ocp.get_t()

# State space expression.
f = ocp.get_zero_vector(n_x)
A = np.array([[1.1, 2.0],
              [0, 0.95]])
B = np.array([[0.0],
              [0.0787]])
# continuous dynamics.
# A = np.array([[1.0, 20],
#               [0.0, -0.5]])
# B = np.array([[0.0],
#               [0.787]])
C = np.array([[-2.0, 1.0]])
f = A * x + B * u

# cost function
Q = C.T @ C
R = np.eye(1)
l =  0.5 * (x.T * Q * x + u.T * R * u )
lf = 0.5 * (x.T * Q * x)

# initial condition.
t0 = 0.0
x0 = np.array([3.0, 1.0])

# horizon length and discretization grids.
T = 3.0
N = 30

# define ocp
ocp.define_unconstrained(f, l, lf, t0, x0, T, N, is_continuous=False)

# solver
# solver = DDPSolver(ocp)
solver = iLQRSolver(ocp)

# initial guess
us_guess = np.zeros((N, n_u))
solver.set_guess(us_guess=us_guess)


In [ ]:
# set parameters
solver.set_regularization_coeff(
    gamma_init=0.0, rho_gamma=1.0, gamma_min=0.0, gamma_max=0.0
)

# solve
xs, us, ts, is_success = solver.solve(
    result=True, log=True, plot=True
)


In [ ]:
print('xs\n', xs)
print('us\n', us)

In [ ]:
import matplotlib.pyplot as plt
import math

result = solver.get_result()

gamma_hist = result['gamma_hist']
plt.plot(gamma_hist)
plt.title('gamma')
plt.show()

alpha_hist = result['alpha_hist']
plt.plot(alpha_hist)
plt.title('alpha')
plt.show()
print('average alpha:',sum(alpha_hist / (len(alpha_hist) - 1)))

cost_hist = result['cost_hist']
plt.plot(cost_hist)
plt.title('cost')
plt.show()
